In [23]:
import pandas as pd
import matplotlib.pyplot as plt
import re

In [4]:
def fix_price(price):
    if '€' in price:
        return float(price.replace('€', ''))
    elif 'lei' in price:
        return float(price.replace('lei', ''))/4.5
    elif 'Schimb' in price:
        return float('nan')
    raise Exception(price)

estates = pd.read_json("../olx/houses.jl", lines=True)
estates["price"] = estates["price"].map(fix_price)

estates.loc[(estates["type"] != "Case de vanzare Oradea") 
            & (estates["type"] != "Case de inchiriat Oradea") 
            & (estates["price"] > 4000), "type"] = "apartament_vanzare"
estates.loc[(estates["type"] != "Case de vanzare Oradea") 
            & (estates["type"] != "Case de inchiriat Oradea") 
            & (estates["price"] <= 4000), "type"] = "apartament_inchiriat"
estates.loc[~estates["type"].isin({"Case de vanzare Oradea", "Case de inchiriat Oradea", 
                                     "apartament_vanzare", "apartament_inchiriat"}), "type"] = "apartament_vanzare"

In [5]:
estates.head()

,An constructie,Camere,Compartimentare,Etaj,Locuinta mobilata / utilata,Oferit de,Suprafata utila,adaugat_la,date,nr_anunt,price,text,title,type,url
0,NaN,4 sau mai multe,NaN,NaN,Partial,Proprietar,215 m²,"[11:00, 18 ianuarie 2018]",2018-02-07,[130755631],65000.0,"CASA de vanzare DRAGESTI 25 Km de oradea,Casa...",Casa de vanzare sau schimb cu apartament sau t...,Case de vanzare Oradea,https://www.olx.ro/oferta/casa-de-vanzare-sau-...
1,NaN,4 sau mai multe,NaN,NaN,NaN,Agentie,200 m²,"[16:58, 7 februarie 2018]",2018-02-07,[131655047],245000.0,"Gaminvest propune spre vanzare, vila D+P+1 cu ...","Vila de vanzare in Oncea, Oradea, Bihor V1322",Case de vanzare Oradea,https://www.olx.ro/oferta/vila-de-vanzare-in-o...
2,NaN,4 sau mai multe,NaN,NaN,Complet,Agentie,145 m²,"[17:04, 7 februarie 2018]",2018-02-07,[113666740],95000.0,"Casa 4 camere de vanzare, Oradea zona ultracen...","Casa 4 camere de vanzare, Oradea zona ultracen...",Case de vanzare Oradea,https://www.olx.ro/oferta/casa-4-camere-de-van...
3,NaN,4 sau mai multe,NaN,NaN,NaN,Agentie,221 m²,"[17:25, 7 februarie 2018]",2018-02-07,[152522749],105000.0,De vânzare URGENT casa compusa din 2 corpuri c...,"Urgent casa 2 corpuri ,zona Centru Civic",Case de vanzare Oradea,https://www.olx.ro/oferta/urgent-casa-2-corpur...
4,NaN,4 sau mai multe,NaN,NaN,NaN,Proprietar,NaN,[],2018-02-07,[152043909],149000.0,Proprietar vand 2 cladiri cu 24 de apartamente...,Vand in Germania bloc cu 24 de apartamente,Case de vanzare Oradea,https://www.olx.ro/oferta/vand-in-germania-blo...


In [6]:
estates["type"].unique()

array(['Case de vanzare Oradea', 'apartament_vanzare',
       'Case de inchiriat Oradea', 'apartament_inchiriat'], dtype=object)

In [79]:
nb = {"oncea", "iosia", "velenta", "rogerius", "decebal", "nufarul", "centru", "grigorescu", 
     "cantemir", "balcescu", "ultracentrala", "aradului", "central", "republicii", 
     "ultracentral", "nufărul"}
outside = {"haieu", "kismarja", "sanmartin", "remeti", "nojorid", "osorhei", "episcopia", 
           "munteni", "1 mai", "ianosda", "paleu"}


def getNeighborhood(text):
    # TODO: needs to handle multiword names
    text = re.split(" |,|\.|-", text.lower())
    street = False
    zona = False
    for t in text:
        if street and t!="":
            return "Strada " + t
        if zona and t != "":
            return "Zona " + t
        if t in nb:
            return t
        if t in outside:
            return "Outside"
        if t == "str" or t == "strada":
            street = True
        if t == "zona" or t == "cartier":
            zona = True
    return ''
        
        
estates["neighborhood"] = estates["title"].map(getNeighborhood)

In [80]:
pd.set_option('display.max_colwidth', -1)
list(estates[estates["neighborhood"] == '']["title"])

['Casa de vanzare sau schimb cu apartament sau teren Oradea',
 'Vand in Germania bloc cu 24 de apartamente',
 'De vanzare casa / vila in Oradea, Bihor, Romania  V0710',
 'Inchiriez camere la casa',
 'Închiriez vilă',
 'Inchiriez vila/pensiune Baile 1 Mai',
 'Casa D+P+M,3 dorm,315 mp,Semicentrala,finisata lux,1200E',
 'Apartament 3 camere, Ared Lidl',
 'Apartament 2 camere de inchiriat',
 'Dau în chirie casa',
 'Casa 2 camere',
 'De închiriat ! Apartament cu 3 camere - cartierul Prima',
 'Teren cu vad comercial - DE INCHIRIAT',
 'Apartament cu 2 camere de inchiriat!',
 'Chirie Casa',
 'Dau în chirie salon coafura canina',
 'Apartament  2 camere nou,lux Ared Kaufland',
 'Apartament cu 2 camere de inchiriat, Cartierul Ared 1 (langa Lidl)',
 'Penthouse de inchiriat – cart. Prima, Oradea',
 'Apartament 2 camere de inchiriat, cart. Prima – Oradea',
 'OCAZIE!!Dau in chirie apartament 2 camere- Facultatea de  Medicina',
 'De inchiriat apartament cu 2 camere lux, bloc Ared, Oradea A1033',
 'Inc

In [78]:
estates[estates["neighborhood"] != ""].count()

An constructie                 499 
Camere                         496 
Compartimentare                719 
Etaj                           656 
Locuinta mobilata / utilata    276 
Oferit de                      1301
Suprafata utila                1242
adaugat_la                     1301
date                           1301
nr_anunt                       1301
price                          1296
text                           1301
title                          1301
type                           1301
url                            1301
neighborhood                   1301
dtype: int64